In [1]:
import os
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
import pandas as pd


In [2]:
# Carica il modello ResNet50 pre-addestrato
model = models.resnet50(pretrained=True)
model.eval()  # Mette il modello in modalità di valutazione

/mnt/c/Users/Telemaco/Desktop/elab/vipm_project/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/c/Users/Telemaco/Desktop/elab/vipm_project/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
# Funzione per caricare e preparare l'immagine
def load_and_prepare_image(image_path):
    img = Image.open(image_path)  # Carica l'immagine
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])  # Trasformazioni necessarie
    
    img_tensor = transform(img)
    img_tensor = img_tensor.unsqueeze(0)  # Aggiungi la dimensione del batch
    return img_tensor

# Funzione per classificare una serie di immagini
def classify_images_in_directory(directory_path, file_names):
    results = []
    for img_file in file_names:
        #img_path = os.path.join(directory_path, './' + img_file)  # Costruisci il percorso completo
        img_tensor = load_and_prepare_image(directory_path + '/' + img_file)  # Prepara l'immagine
        
        # Previsioni sul modello
        with torch.no_grad():
            outputs = model(img_tensor)
        
        _, predicted_class = torch.max(outputs, 1)  # Trova la classe con il punteggio più alto
        
        results.append(predicted_class.item())
        # Stampa il risultato per questa immagine
        print(f"Image: {img_file} - Predected class ID: {predicted_class.item()}")

    return results


In [5]:
# Percorso della directory contenente le immagini
directory_path = "../dataset/small_dataset"  # Sostituisci con il percorso della tua cartella

data_train_labelled = pd.read_csv("../dataset/train_small.csv", header=None)
#print(list(data_train_labelled[0]))
# Classifica le immagini
results = classify_images_in_directory(directory_path, list(data_train_labelled[0])[:20])


Image: train_059371.jpg - Predected class ID: 116
Image: train_059389.jpg - Predected class ID: 969
Image: train_059425.jpg - Predected class ID: 943
Image: train_059445.jpg - Predected class ID: 470
Image: train_059516.jpg - Predected class ID: 429
Image: train_059538.jpg - Predected class ID: 551
Image: train_059541.jpg - Predected class ID: 923
Image: train_059570.jpg - Predected class ID: 551
Image: train_059713.jpg - Predected class ID: 551
Image: train_059719.jpg - Predected class ID: 398
Image: train_059744.jpg - Predected class ID: 930
Image: train_059789.jpg - Predected class ID: 522
Image: train_059795.jpg - Predected class ID: 960
Image: train_059837.jpg - Predected class ID: 631
Image: train_059855.jpg - Predected class ID: 470
Image: train_059893.jpg - Predected class ID: 767
Image: train_059917.jpg - Predected class ID: 948
Image: train_059919.jpg - Predected class ID: 940
Image: train_059921.jpg - Predected class ID: 463
Image: train_059928.jpg - Predected class ID: 551


In [6]:
results

[116,
 969,
 943,
 470,
 429,
 551,
 923,
 551,
 551,
 398,
 930,
 522,
 960,
 631,
 470,
 767,
 948,
 940,
 463,
 551]